In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.feature_extraction import text  
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import sqlite3
cnx = sqlite3.connect('db.sqlite3')
df = pd.read_sql_query("SELECT * FROM application_data_set",cnx)
df.set_index('id')

def build_train_test():
    
    # Create your connection.
    
    #data = pd.read_csv('spam.csv') 
    #data = data[['Label','EmailText']]
    #data.dropna(axis='index',how='any',inplace=True)
    
    train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

    return train_set['EmailText'],train_set['Label'],test_set['EmailText'],test_set['Label']

In [2]:

X_train,Y_train,X_test,Y_test = build_train_test()
vectorizer = text.CountVectorizer(stop_words="english")
counts = vectorizer.fit_transform(X_train.values)
spam_nb = MultinomialNB()
spam_nb.fit(counts,Y_train.values)

MultinomialNB()

In [3]:
def model_accuracy():
    examples = vectorizer.transform(X_test.values)
    predicted = spam_nb.predict(examples)
    prediction_bool = pd.Series(predicted==Y_test.values)
    true_count = prediction_bool[prediction_bool==True].count()
    false_count = prediction_bool[prediction_bool==False].count()
    print((100*true_count)/(true_count+false_count))

In [4]:
def test_model(EmailText):
    test_vect = vectorizer.transform(np.array([EmailText]))
    predict_test_vect = spam_nb.predict(test_vect)
    print(predict_test_vect[0])
    #return predict_test_vect[0]
    global df
    new_row = {'id': df.shape[0] + 1,'Label':predict_test_vect[0],'EmailText':EmailText}
    df = df.append(new_row, ignore_index=True)

In [5]:
test_model('Sale Sale Sale Get 40% off Now')

spam


In [6]:
model_accuracy()

98.74776386404294


In [7]:
#df.to_sql(name='testdb',con=cnx)

ValueError: Table 'testdb' already exists.

In [8]:
df

,id,Label,EmailText
0,1,Label,EmailText
1,2,ham,"Go until jurong point, crazy.. Available only ..."
2,3,ham,Ok lar... Joking wif u oni...
3,4,spam,Free entry in 2 a wkly comp to win FA Cup fina...
4,5,ham,U dun say so early hor... U c already then say...
...,...,...,...
5585,5586,spam,Want explicit SEX in 30 secs? Ring 02073162414...
5586,5587,ham,I can't believe how attached I am to seeing yo...
5587,5588,ham,Just sleeping..and surfing
5588,5589,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
